In [157]:
from keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv2D, Flatten, concatenate, Input, Dropout, MaxPooling2D
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
from TextAudioMultimodalClassifier7 import TextAudioMultimodalClassifier
import pickle
from torch.optim.lr_scheduler import StepLR

## 데이터 전처리

In [92]:
# 1~6까지
n = "6"

In [27]:
with open('D:\\text_audio_features\\기쁨\\textF' + n + '.pkl', 'rb') as file:
    happy_textF = pickle.load(file)

with open('D:\\text_audio_features\\슬픔\\textF' + n + '.pkl', 'rb') as file:
    sad_textF = pickle.load(file)
    
with open('D:\\text_audio_features\\기쁨\\textF' + "7" + '.pkl', 'rb') as file:
    happy_textF7 = pickle.load(file)

with open('D:\\text_audio_features\\슬픔\\textF' + "7" + '.pkl', 'rb') as file:
    sad_textF7 = pickle.load(file)

with open('D:\\text_audio_features\\분노\\textF' + n + '.pkl', 'rb') as file:
    angry_textF = pickle.load(file)

with open('D:\\text_audio_features\\중립\\textF' + n + '.pkl', 'rb') as file:
    normal_textF = pickle.load(file)
    

In [24]:
textF = np.concatenate([happy_textF, sad_textF, angry_textF, normal_textF])
textF = textF.astype('float32')
len(textF)

39901

In [28]:
textF = np.concatenate([happy_textF, happy_textF7, sad_textF, sad_textF7, angry_textF, normal_textF])
textF = textF.astype('float32')
len(textF)

57302

In [29]:
textF6 = textF[:]

In [30]:
textFF = np.concatenate([textF1, textF2, textF3, textF4, textF5, textF6])

260171개의 데이터셋

In [31]:
len(textFF)

260171

In [93]:
with open('D:\\text_audio_features\\기쁨\\audioF' + n + '.pkl', 'rb') as file:
    happy_audioF = pickle.load(file)

with open('D:\\text_audio_features\\슬픔\\audioF' + n + '.pkl', 'rb') as file:
    sad_audioF = pickle.load(file)

with open('D:\\text_audio_features\\기쁨\\audioF' + "7" + '.pkl', 'rb') as file:
    happy_audioF7 = pickle.load(file)

with open('D:\\text_audio_features\\슬픔\\audioF' + "7" + '.pkl', 'rb') as file:
    sad_audioF7 = pickle.load(file)

with open('D:\\text_audio_features\\분노\\audioF' + n + '.pkl', 'rb') as file:
    angry_audioF = pickle.load(file)

with open('D:\\text_audio_features\\중립\\audioF' + n + '.pkl', 'rb') as file:
    normal_audioF = pickle.load(file)

In [94]:
audioF = np.concatenate([happy_audioF, sad_audioF, angry_audioF, normal_audioF])
audioF = audioF.astype('float32')
len(audioF)

40443

In [54]:
audioF = np.concatenate([happy_audioF, happy_audioF7, sad_audioF, sad_audioF7, angry_audioF, normal_audioF])
audioF = audioF.astype('float32')
len(audioF)

57302

In [95]:
audioF4 = audioF[:]

In [96]:
audioFF = np.concatenate([audioF1, audioF2, audioF3, audioF4, audioF5, audioF6])

In [97]:
len(audioFF)

260171

In [98]:
with open('D:\\text_audio_features\\기쁨\\y' + n + '.pkl', 'rb') as file:
    happy_y = pickle.load(file)

with open('D:\\text_audio_features\\슬픔\\y' + n + '.pkl', 'rb') as file:
    sad_y = pickle.load(file)

with open('D:\\text_audio_features\\기쁨\\y' + "7" + '.pkl', 'rb') as file:
    happy_y7 = pickle.load(file)

with open('D:\\text_audio_features\\슬픔\\y' + "7" + '.pkl', 'rb') as file:
    sad_y7 = pickle.load(file)
    
with open('D:\\text_audio_features\\분노\\y' + n + '.pkl', 'rb') as file:
    angry_y = pickle.load(file)

with open('D:\\text_audio_features\\중립\\y' + n + '.pkl', 'rb') as file:
    normal_y = pickle.load(file)

In [99]:
y = np.concatenate([happy_y, sad_y, angry_y, normal_y])
len(y)

40443

In [80]:
y = np.concatenate([happy_y, happy_y7, sad_y, sad_y7, angry_y, normal_y])
len(y)

57302

In [100]:
y4 = y[:]

In [102]:
yy = np.concatenate([y1, y2, y3, y4, y5, y6])

In [103]:
len(yy)

260171

In [104]:
class CustomDataset(Dataset):
    def __init__(self, text_features, audio_features, labels):
        self.text_features = text_features
        self.audio_features = audio_features
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        text_input = torch.tensor(self.text_features[index], dtype=torch.float32)
        audio_input = torch.tensor(self.audio_features[index], dtype=torch.float32)
        label = torch.tensor(self.labels[index], dtype=torch.long)
        
        return text_input, audio_input, label


# 데이터로더 생성
batch_size = 512  # 배치 크기 설정

In [105]:
# 데이터를 훈련 데이터와 검증 데이터로 나눔
train_textF, val_textF, train_audioF, val_audioF, train_y, val_y = train_test_split(
    textFF, audioFF, yy, test_size=0.2, random_state=42)

# 훈련 데이터셋 인스턴스 생성
train_dataset = CustomDataset(text_features=train_textF, audio_features=train_audioF, labels=train_y)

# 검증 데이터셋 인스턴스 생성
val_dataset = CustomDataset(text_features=val_textF, audio_features=val_audioF, labels=val_y)

# 데이터로더 생성 (훈련 데이터용)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 데이터로더 생성 (검증 데이터용)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## 모델 훈련

In [158]:
# 모델 초기화
input_dim_text = 768  # 텍스트 특징 개수
input_dim_audio = 87  # 오디오 특징 개수
num_classes = 7  # 감정 클래스 개수

model = TextAudioMultimodalClassifier(input_dim_text, input_dim_audio, num_classes)

'''# X에 있는 임베딩 값을 모델에 입력으로 사용
with torch.no_grad():
    outputs = model(torch.tensor(X))'''

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# 조기 종료를 위한 변수 초기화
best_val_loss = float('inf')
patience = 5
early_stopping_counter = 0

# 모델 훈련
num_epochs = 10000

# 모델을 장치로 이동 (CPU 또는 GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

TextAudioMultimodalClassifier(
  (text_layer): Linear(in_features=768, out_features=512, bias=True)
  (audio_layer): Linear(in_features=87, out_features=512, bias=True)
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=512, out_features=7, bias=True)
)

In [159]:
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    
    for text_input, audio_input, labels in train_dataloader:  # train_dataloader에 대한 반복
        text_input = text_input.to(device)
        audio_input = audio_input.to(device)
        labels = labels.to(device).argmax(dim=1)
        
        optimizer.zero_grad()  # 그레이디언트 초기화
        
        # 모델 예측
        outputs = model(text_input, audio_input)
        
        # 손실 계산
        loss = criterion(outputs, labels)
        
        # 역전파 및 가중치 업데이트
        loss.backward()
        optimizer.step()
    
    # 스케줄러 업데이트
    scheduler.step()
    
    # 검증 데이터셋을 사용하여 모델 평가
    model.eval()  # 모델을 평가 모드로 설정
    total_correct = 0
    total_samples = 0
    val_loss = 0
    
    with torch.no_grad():
        val_loss = 0.0
        for text_input, audio_input, labels in val_dataloader:  # val_dataloader에 대한 반복
            text_input = text_input.to(device)
            audio_input = audio_input.to(device)
            labels = labels.to(device).argmax(dim=1)
            
            outputs = model(text_input, audio_input)
            probabilities = F.softmax(outputs, dim=1)  # 소프트맥스 함수를 적용
            _, predicted = torch.max(probabilities, dim=1)  # 확률값 중에서 가장 큰 값의 인덱스를 선택
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            
            val_loss += criterion(outputs, labels).item()
            val_loss += criterion(outputs, labels).item() * text_input.size(0)

    accuracy = total_correct / total_samples
    val_loss /= len(val_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}, Validation Loss: {val_loss:.4f}")

    # 조기 종료 확인
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        print("up")
        if early_stopping_counter >= patience:
            print("Early stopping triggered!")
            break

Epoch [1/10000], Accuracy: 0.4056, Validation Loss: 1.3315
Epoch [2/10000], Accuracy: 0.4185, Validation Loss: 1.2877
Epoch [3/10000], Accuracy: 0.4337, Validation Loss: 1.2691
Epoch [4/10000], Accuracy: 0.4438, Validation Loss: 1.2559
Epoch [5/10000], Accuracy: 0.4507, Validation Loss: 1.2452
Epoch [6/10000], Accuracy: 0.4558, Validation Loss: 1.2365
Epoch [7/10000], Accuracy: 0.4603, Validation Loss: 1.2292
Epoch [8/10000], Accuracy: 0.4633, Validation Loss: 1.2235
Epoch [9/10000], Accuracy: 0.4650, Validation Loss: 1.2182
Epoch [10/10000], Accuracy: 0.4666, Validation Loss: 1.2136
Epoch [11/10000], Accuracy: 0.4705, Validation Loss: 1.2095
Epoch [12/10000], Accuracy: 0.4725, Validation Loss: 1.2055
Epoch [13/10000], Accuracy: 0.4740, Validation Loss: 1.2018
Epoch [14/10000], Accuracy: 0.4754, Validation Loss: 1.1983
Epoch [15/10000], Accuracy: 0.4734, Validation Loss: 1.1960
Epoch [16/10000], Accuracy: 0.4785, Validation Loss: 1.1930
Epoch [17/10000], Accuracy: 0.4794, Validation Lo

52%의 정확도

## 모델 저장

In [109]:
save_path = "model\\text_audio_model_52.pth"

torch.save(model.state_dict(), save_path)